# Automatic averaging of pilatus frames in timescans

This notebook will routinely look for new files with timescans and average the pilatus frames in each timescan.

The averaged frame are saved as `.tiff` images in `/data/visitors/danmax/PROPOSAL/VISIT/process/average/SAMPLE/`

The required input is the sample folder where the new files are stored.

To stop the process choose `Kernel`>`Restart Kernel and Clear All Outputs...`

In [ ]:
pname = '/data/visitors/danmax/PROPOSAL/VISIT/raw/SAMPLE/'

#------------------------------------------------------------------------

%matplotlib widget
import os.path
import sys
import glob
import time

import h5py
import numpy as np

import tifffile

if not os.path.isdir(pname):
    print('The specified directory does not exit - please start the experiment first or check the specified path')
    sys.exit(0)

#Create folder for data if it does not exist
out_path = pname.replace('raw', 'process/average')
if not os.path.exists(out_path):
    print(f'Creating folder: {out_path}')
    os.makedirs(out_path)
    
while True: 
    fnames = glob.glob(pname+'/*.h5', recursive=False)
    for fname in fnames:
        # Do not consider pilatus data files
        # Do not consider falconx data files
        # Skip if average file exist
        if '_pilatus.h5' not in fname and '_falconx.h5' not in fname and not os.path.isfile(fname.split('.')[0].replace('raw', 'process/average')+'_averaged.tiff'): 
            if os.path.isfile(fname.split('.')[0]+'_pilatus.h5'): # Check only if pilatus data is available 
                try:
                    fh = h5py.File(fname, 'r')
                    scanCmd = fh['entry/title'][()].decode('utf-8').split()
                    if scanCmd[0] == 'timescan': #Proces only if it is a timescan
                        try:
                            images = fh['/entry/instrument/pilatus/data']
                            avg_img = np.mean(images, axis=0)
                            tifffile.imwrite(fname.split('.')[0].replace('raw', 'process/average')+'_averaged.tiff', avg_img[np.newaxis,:].astype(np.float32))
                            print("Averaged frames saved as {0}".format(fname.split('.')[0].replace('raw', 'process/average')+'_averaged.tiff'))
                        except KeyError:
                            pass
                except OSError: # Ignore if file is not readable (e.g. while it is being written...)
                    pass
    # Write output while waiting for next check for new files
    for i in range(5):
        print("Will look for new files in {0} seconds".format(5-i), end='\r')
        time.sleep(1)

In [ ]:

#Create a distination folder for data if it does not exist
dst = os.getcwd.replace('raw', 'process/average')
if not os.path.exists(dst):
    print(f'Creating folder: {dst}')
    os.makedirs(dst)
    
scan_type = 'any' # timescan, dscan, ascan, meshct, etc.

scans = DM.findAllScans(descending=False)
if not scan_type == 'any':
    # sort for the specified scan type
    scans = [scan for scan in scans if scan_type in DM.getScanType(scan)]
# remove scans with already existing average tiff images
scans = [scan for scan in scans if not os.path.isfile(scan.replace('raw', 'process/average').replace('.h5','_averaged.tiff' ))]

im = DM.getAverageImage(fname)